In [11]:
import sqlite3
import nest_asyncio
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove
from aiogram.dispatcher import FSMContext
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.contrib.fsm_storage.memory import MemoryStorage
bot = Bot('7629247815:AAFDwohea1Y3blp8eHo1frRdIbKN6LVMN_s')
nest_asyncio.apply()

In [ ]:
class Mystate(StatesGroup):
    register = State()
    nextstep = State()



storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)


# Connection to database and creating database
conn = sqlite3.connect('database.sql')
cur = conn.cursor()

cur.execute('CREATE TABLE IF NOT EXISTS clients (id INTEGER PRIMARY KEY, name TEXT, email TEXT)')
conn.commit()


# Button: Start
@dp.message_handler(commands=['start'])
async def start(message: types.Message):
    await message.answer('''Hello, to register please enter your full name and email: 
    Name: <your name>
    Email: <your email>''')
    await Mystate.register.set()


# Info about user goes to database
@dp.message_handler(state=Mystate.register)
async def register_user(message: types.Message, state: FSMContext):
    data = message.text.split('\n')
    if len(data) >= 2:
        try:
            name = data[0].split(': ')[1]
            email = data[1].split(': ')[1]

            markup = ReplyKeyboardMarkup(resize_keyboard=True)
            markup.add(KeyboardButton('/Next_step'))

            cur.execute("INSERT INTO clients (name, email) VALUES (?, ?)", (name, email))
            conn.commit()

            await message.answer(f'You are registered as {name}', reply_markup=markup)
            await Mystate.nextstep.set()
        except Exception as e:
            await message.answer(f'Error: {e}')
    else:
        await message.answer('Invalid input format. Please enter your name and email in the correct format.')


# Button: Next step
@dp.message_handler(state=Mystate.nextstep, commands=['Next_step'])
async def next_step(message: types.Message, state: FSMContext):
    data = message.text.split()
    
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Yes'),
              Keyboardbutton('/No'))
    
    await message.answer(
        '''
        Hello again, you wrote to a telegram bot that can predict your risk of cardiovascular disease based on your specific readings, are you interested in this?
        ''', reply_markup=markup)
    await state.finish()


executor.start_polling(dp) #обязательная для работы бота часть"